# Cobranza (Recuperación de Cartera)

Notebook para predecir la probabilidad de recuperación de deudas usando datos sintéticos o CSV externo.

## 1. Importar librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

## 2. Cargar datos
### A) Generar datos sintéticos

In [ ]:
np.random.seed(42)
n = 1000
data = pd.DataFrame({
    'dias_atraso': np.random.randint(0, 120, n),
    'monto_deuda': np.random.uniform(100, 10000, n),
    'pagos_previos': np.random.randint(0, 5, n),
    'contactos': np.random.randint(0, 10, n),
    'segmento': np.random.choice(['Retail','Pyme','Corporativo'], n)
})
data['pago'] = (0.2*(data['dias_atraso']<30) + 0.2*(data['pagos_previos']>1) + 0.1*(data['contactos']>2) + np.random.rand(n)) > 0.5
data['pago'] = data['pago'].astype(int)
data.head()

### B) Cargar datos desde CSV
> Descomenta y ajusta la ruta si tienes tu propio archivo.

In [ ]:
# data = pd.read_csv('deudas_cobranza.csv')
# data.head()

## 3. Análisis exploratorio

In [ ]:
print(data['pago'].value_counts())
data.describe()
data.groupby('segmento')['pago'].mean().plot(kind='bar')
plt.title('Tasa de pago por segmento')
plt.ylabel('Pago Rate')
plt.show()

## 4. Preprocesamiento

In [ ]:
X = data.drop('pago', axis=1)
y = data['pago']
X['segmento'] = LabelEncoder().fit_transform(X['segmento'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

## 5. Modelos
### A) Regresión Logística

In [ ]:
lr = LogisticRegression(max_iter=200)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print(classification_report(y_test, y_pred_lr))

### B) XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

## 6. Evaluación y visualización

In [ ]:
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plt.title('Confusión LR')
plt.imshow(confusion_matrix(y_test, y_pred_lr), cmap='Blues')
plt.subplot(1,2,2)
plt.title('Confusión XGBoost')
plt.imshow(confusion_matrix(y_test, y_pred_xgb), cmap='Greens')
plt.show()
fpr_lr, tpr_lr, _ = roc_curve(y_test, lr.predict_proba(X_test)[:,1])
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, xgb_model.predict_proba(X_test)[:,1])
plt.plot(fpr_lr, tpr_lr, label='LogisticReg')
plt.plot(fpr_xgb, tpr_xgb, label='XGBoost')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()
plt.title('Curvas ROC')
plt.show()

## 7. Importancia de variables
### XGBoost

In [ ]:
xgb.plot_importance(xgb_model)
plt.show()

## 8. Comentarios finales
- Puedes modificar los parámetros, probar con CSV propio y agregar nuevas visualizaciones.